In [ ]:
# Import necessary libraries
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import joblib
from tkinter import filedialog
from IPython.display import display
from PIL import Image

In [ ]:
# Updated folder structure mapping
LABELS = {
    'in-range-light': 1,
    'in-range-dark': 1,
    'in-range-normal': 1,
    'out-of-range-too-light': 0,
    'out-of-range-too-dark': 0,
    'out_range': 0
}

# Function to apply Lab conversion and CLAHE
def extract_features(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return None
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    
    # Apply CLAHE to the L channel
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    lab[:,:,0] = clahe.apply(lab[:,:,0])
    
    # Extract statistical features
    l_mean, a_mean, b_mean = lab[:,:,0].mean(), lab[:,:,1].mean(), lab[:,:,2].mean()
    l_std, a_std, b_std = lab[:,:,0].std(), lab[:,:,1].std(), lab[:,:,2].std()
    
    return [l_mean, a_mean, b_mean, l_std, a_std, b_std]

# Load dataset based on new folder structure
def load_dataset(base_dir='images'):
    X, y = [], []
    for folder, label in LABELS.items():
        folder_path = os.path.join(base_dir, folder)
        if not os.path.exists(folder_path):
            print(f"Warning: Missing folder {folder_path}")
            continue
        for file in tqdm(os.listdir(folder_path), desc=f"Loading {folder}"):
            img_path = os.path.join(folder_path, file)
            features = extract_features(img_path)
            if features:
                X.append(features)
                y.append(label)
    return np.array(X), np.array(y)

In [ ]:
# Load and preprocess dataset
X, y = load_dataset()

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train multiple models
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM (RBF Kernel)": SVC(kernel='rbf', probability=True, random_state=42),
    "MLP Neural Network": MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
}

In [ ]:
model_performance = {}

for name, model in models.items():
    print(f"\nTraining {name} model...")
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    
    # Evaluate and store performance
    acc = accuracy_score(y_test, y_pred)
    model_performance[name] = acc
    print(f"{name} Accuracy: {acc*100:.2f}%")
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    
    # Save the model
    joblib.dump(model, f"{name.replace(' ', '_')}_model.pkl")

# Save scaler
joblib.dump(scaler, 'graphite_walnut_scaler.pkl')

# Display model performance comparison
print("\nModel Performance Summary:")
for name, acc in model_performance.items():
    print(f"{name}: {acc*100:.2f}%")

print("Models and scaler saved.")

In [ ]:
def upload_image():
    """ Open file dialog for image upload """
    file_path = filedialog.askopenfilename(title="Select Image", filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
    return file_path

def validate_sample(model_name):
    print(f"\nUsing {model_name} model for validation...")
    print("Upload the known in-range sample:")
    ref_path = upload_image()
    ref_features = extract_features(ref_path)
    if ref_features is None:
        print("Invalid reference image. Please try again.")
        return
    
    print("Upload the sample to be validated:")
    sample_path = upload_image()
    sample_features = extract_features(sample_path)
    if sample_features is None:
        print("Invalid sample image. Please try again.")
        return
    
    # Normalize features relative to reference sample
    scaler = joblib.load('graphite_walnut_scaler.pkl')
    model = joblib.load(f"{model_name.replace(' ', '_')}_model.pkl")
    
    # Compute relative features
    normalized_sample = np.array(sample_features) - np.array(ref_features)
    normalized_sample = scaler.transform([normalized_sample])
    
    prediction = model.predict(normalized_sample)[0]
    result = "In Range" if prediction == 1 else "Out of Range"
    
    print(f"Sample Validation Result: {result}")

# Choose model for validation
print("\nSelect the model you want to use for validation:")
for i, name in enumerate(models.keys()):
    print(f"{i+1}. {name}")

choice = int(input("Enter the number corresponding to the model: ")) - 1
selected_model = list(models.keys())[choice]

# Run validation with the selected model
validate_sample(selected_model)